# Tabular Models

**SDV** supports modeling single table datasets. It provides unique features for making it easy
for the user to learn models and synthesize datasets. Some important features of `sdv.tabular` include:

* Support for tables with primary key
* Support to anonymize certain fields like addresses, emails, phone numbers, names and other
  PII information.
* Support for a number of different data types - categorical, numerical, discrete-ordinal
  and datetimes.
* Support multiple types of statistical and deep learning models:
  * GaussianCopula: A tool to model multivariate distributions using [copula functions](
    https://en.wikipedia.org/wiki/Copula_%28probability_theory%29). Based on our [Copulas Library](
    https://github.com/sdv-dev/Copulas).
  * CTGAN: A GAN-based Deep Learning data synthesizer that can generate synthetic tabular data with high 
    fidelity. Based on our [CTGAN Library](https://github.com/sdv-dev/CTGAN).

In this guide we will go through a series of steps that will let you discover the common
functionalities of the Tabular Models, including how to:

* Create an instance of a Tabular Model.
* Fit the instance to your data.
* Generate synthetic versions of your data.
* Use the a Tabular Model to anonymize PII information.
* Customize the data tranformations to improve the learning process.
* Specify inter-column constraints to improve the output quality.

<div class="alert alert-info">

**NOTE**

During this guide we will use the `GaussianCopula` model as an example but we
will be talking only about the functionalities that are shared across all the Tabular Models.
To discover specific features of the GaussianCopula in particular please visit the
[GaussianCopula User Guide](02_GaussianCopula_Model.ipynb).

</div>

## Load and explore tabular data

We will start by loading one of our demo datasets, the `student_placements`, which contains information
about MBA students that applied for placements during the year 2020.

In [1]:
# Setup logging and warnings - change ERROR to INFO for increased verbosity
import logging;
logging.basicConfig(level=logging.ERROR)

logging.getLogger().setLevel(level=logging.WARNING)
logging.getLogger('sdv').setLevel(level=logging.ERROR)

import warnings
warnings.simplefilter("ignore")

In [2]:
from sdv.demo import load_tabular_demo

data = load_tabular_demo('student_placements')
data.head().T

,0,1,2,3,4
student_id,155368,155369,155370,155371,155372
gender,M,M,M,M,M
second_perc,67,79.33,65,56,85.8
high_perc,91,78.33,68,52,73.6
high_spec,Commerce,Science,Arts,Science,Commerce
degree_perc,58,77.48,64,52,73.3
degree_type,Sci&Tech,Sci&Tech,Comm&Mgmt,Sci&Tech,Comm&Mgmt
work_experience,False,True,False,False,False
experience_years,0,1,0,0,0
employability_perc,55,86.5,75,66,96.8


As you can see, this table contains information about students which includes, among other things:

- Their id and gender
- Their grades and specializations
- Their work experience
- The salary that they where offered
- The duration and dates of their placement

You will notice that there is data with the following characteristics:

- There are float, integer, boolean, categorical and datetime values.
- There are some variables that have missing data. In particular, all the data related to the
  placement details is missing in the rows where the studen was not placed.

Let us use the `GaussianCopula` to learn this data and then sample synthetic data about new students
to see how well de model captures the characteristics indicated above.

In order to do this you will need to:

- Import the `sdv.tabular.GaussianCopula` class and create an instance of it.
- Call its `fit` method passing our table.
- Call its `sample` method indicating the number of synthetic rows that you want to generate.

In [3]:
from sdv.tabular import GaussianCopula

model = GaussianCopula()
model.fit(data)

<div class="alert alert-info">

**NOTE**

Notice that the model `fitting` process took care of transforming the different fields using the
appropriate [Reversible Data Transforms](http://github.com/sdv-dev/RDT) to ensure that the data has
a format that the GaussianMultivariate model from the [copulas](https://github.com/sdv-dev/Copulas)
library can handle.

</div>

## Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic data by calling the `sample` method
from your model passing the number of rows that we want to generate.

In [4]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted on, but filled with new data
which resembles the original one.

In [5]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,155417,F,58.848130,60.248975,Commerce,53.125903,Sci&Tech,False,0,52.044813,Mkt&Fin,58.476942,46048.581084,True,2020-01-06 12:35:56.536872704,2020-06-30 20:51:37.122674176,3.0
1,155538,F,68.438562,68.928067,Science,66.369985,Sci&Tech,False,-1,66.771892,Mkt&HR,57.947703,30979.983151,True,2020-01-04 15:57:33.060699136,2021-01-07 06:14:16.852958720,12.0
2,155560,M,68.726441,72.474372,Commerce,55.974080,Comm&Mgmt,False,1,66.006570,Mkt&HR,63.455531,24842.286502,True,2020-10-11 23:23:38.246976512,2021-01-04 17:33:56.483195904,3.0
3,155406,F,85.530521,86.094248,Science,69.583043,Comm&Mgmt,False,0,68.556037,Mkt&HR,65.335768,28112.501738,True,2020-03-30 15:41:52.219824896,2021-01-15 19:08:38.734086400,12.0
4,155438,M,78.127640,79.473957,Science,73.381697,Comm&Mgmt,False,1,58.082379,Mkt&Fin,76.427970,32819.921441,True,2020-01-25 22:46:28.564114688,2021-01-11 08:11:04.250991872,12.0


<div class="alert alert-info">

**Note**

You can control the number of rows by specifying the number of `samples` in the
`model.sample(<num_rows>)`. To test, try `model.sample(10000)`. Note that the original 
table only had ~200 rows.

</div>

## Save and Load

In many scenarios it will be convenient to generate synthetic versions of your data
directly in systems that do not have access to the original data source.

For example, if you may want to generate testing data on the fly inside a testing
environment that does not have access to your production database.

In these scenarios, fitting the model with real data every time that you need to generate
new data is feasible, so you will need to fit a model in your production environment,
save the fitted model into a file, send this file to the testing environment and then
load it there to be able to `sample` from it.

Let's see how this process works.

### Save and share the model

Once you have fitted the model, all you need to do is call its `save` method passing the
name of the file in which you want to save the model. Note that the extension of the filename
is not relevant, but we will be using the `.pkl` extension to highlight that the serialization
protocol used is [pickle](https://docs.python.org/3/library/pickle.html).

In [6]:
model.save('my_model.pkl')

This will have created a file called `my_model.pkl` in the same directory in which you are
running SDV.

<div class="alert alert-info">

**IMPORTANT**
    
If you inspect the generated file you will notice that its size is much smaller
than the size of the data that you used to generate it. This is because the serialized model
contains **no information about the original data**, other than the parameters it needs to
generate synthetic versions of it. This means that you can safely share this `my_model.pkl`
file without the risc of disclosing any of your real data!
    
</div>

### Load the model and sample from it

The file you just generated can be send over to the system where the synthetic data will be
generated.

Once it is there, you can load it using the `GaussianCopula.load` method, and then you are
ready to sample new data from the loaded instance:

In [7]:
loaded = GaussianCopula.load('my_model.pkl')
new_data = loaded.sample(200)

<div class="alert alert-warning">
    
**WARNING**
    
Notice that the system where the model is loaded needs to also have `sdv` installed,
otherwise it will not be able to load the model and use it.
    
</div>

## Advanced Usage

Now that we have discovered the basics, let's go over a few more advanced usage examples
and see the different arguments that we can pass to our Tabular Models in order to
customize them to our needs.

### Primary Key

One of the first things that you may have noticed when looking that demo data
is that there is a `student_id` column which acts as the primary key of the table,
and which is supposed to have unique values.

Indeed, if we look at the number of times that each value appears, we see that
all of them appear at most once:

In [8]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe that there
are some values that appear more than once:

In [9]:
new_data.student_id.value_counts().max()

5

In [10]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
46,155445,M,78.222479,60.275570,Science,66.266143,Sci&Tech,False,-1,83.893394,Mkt&Fin,59.479104,NaN,False,NaT,NaT,3.0
50,155445,F,69.658239,55.010616,Science,50.425832,Sci&Tech,False,-1,77.397733,Mkt&HR,65.678323,NaN,False,NaT,NaT,12.0
72,155445,F,85.179974,78.274061,Commerce,77.341933,Comm&Mgmt,False,1,58.049310,Mkt&Fin,56.828981,28789.651560,True,2020-01-22 05:09:54.461448192,2020-09-14 14:00:04.591784960,3.0
76,155445,M,72.600745,72.133035,Commerce,71.310510,Comm&Mgmt,False,0,50.489956,Mkt&HR,54.653373,20860.973523,True,2020-01-18 03:51:46.343341056,2020-11-10 05:20:25.560078592,12.0
145,155445,M,62.675550,43.687143,Commerce,58.685724,Sci&Tech,False,1,51.021717,Mkt&HR,58.171704,NaN,False,NaT,NaT,3.0


This happens because the model was not notified at any point about the fact that the
`student_id` had to be unique, so when it generates new data it will provoke collisions
sooner or later.

In order to solve this, we can pass the argument `primary_key` to our model when we create it,
indicating the name of the column that is the index of the table.

In [11]:
model = GaussianCopula(
    primary_key='student_id'
)
model.fit(data)
new_data = model.sample(200)

As a result, the model will learn that this column must be unique and generate a unique
sequence of valures for the column:

In [12]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,M,50.071382,49.652925,Commerce,68.166421,Comm&Mgmt,False,1,63.811532,Mkt&HR,53.177856,24041.369372,True,2020-01-20 18:41:49.069841408,2020-07-18 13:47:11.445067008,3.0
1,1,M,79.737798,74.938517,Commerce,71.569118,Comm&Mgmt,False,0,91.269893,Mkt&Fin,59.599598,33869.082275,True,2020-01-03 18:25:35.109968128,2020-12-24 18:17:19.040092672,12.0
2,2,F,83.937314,70.864656,Science,63.084496,Comm&Mgmt,False,1,59.049658,Mkt&HR,61.443244,23321.557037,True,2021-01-14 16:23:15.729193728,2020-08-11 19:03:28.923697664,3.0
3,3,M,62.121346,74.863317,Commerce,66.404244,Comm&Mgmt,False,1,84.516106,Mkt&Fin,62.650750,NaN,False,NaT,NaT,3.0
4,4,M,83.228037,70.987571,Science,69.160579,Sci&Tech,False,1,91.098498,Mkt&Fin,70.512599,43315.490519,True,2020-01-06 18:45:25.895103232,2020-09-13 06:11:41.222711040,3.0


In [13]:
new_data.student_id.value_counts().max()

1

### Anonymize fields that contain Personally Identifiable Information (PII) 

There will be many cases where the data will contain Personally Identifiable Information
which we cannot disclose.

In these cases, we will want our Tabular Models to replace the information within these
fields with fake, simulated data that looks similar to the real one but does not contain
any of the original values.

Let's load a new dataset that contains a PII field, the `student_placements_pii` demo, and
try to generate synthetic versions of it that do not contain any of the PII fields.

<div class="alert alert-info">
    
**NOTE**
    
The `student_placements_pii` dataset is a modified version of the `student_placements`
dataset with one new field, `name`, which contains PII information about the students.
Notice that this additional `name` field has been simulated and does not correspond to data
from the real users.

</div>

In [14]:
data = load_tabular_demo('student_placements_pii')

In [15]:
data.head().T

,0,1,2,3,4
student_id,155368,155369,155370,155371,155372
name,Ryan Massey,Charles Frye,Corey Compton,John White III,Jeffery Robinson
gender,M,M,M,M,M
second_perc,67,79.33,65,56,85.8
high_perc,91,78.33,68,52,73.6
high_spec,Commerce,Science,Arts,Science,Commerce
degree_perc,58,77.48,64,52,73.3
degree_type,Sci&Tech,Sci&Tech,Comm&Mgmt,Sci&Tech,Comm&Mgmt
work_experience,False,True,False,False,False
experience_years,0,1,0,0,0


If we use our tabular model on this new data we will see how the synthetic
data that it generates discloses the names from the real students:

In [16]:
model = GaussianCopula(
    primary_key='student_id',
)
model.fit(data)

In [17]:
new_data = model.sample(200)
new_data.head()

,student_id,name,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,Andrew Mullins,F,57.424997,59.263592,Science,70.567620,Comm&Mgmt,False,1,93.250983,Mkt&Fin,58.989587,19782.469650,True,2020-01-10 08:15:16.614227968,2020-07-18 11:04:08.337801728,3.0
1,1,Donna Gordon,M,62.708564,55.000428,Arts,62.686418,Comm&Mgmt,True,-1,68.361411,Mkt&Fin,55.430596,NaN,False,NaT,NaT,3.0
2,2,Jeffery Robinson,F,84.241653,72.279247,Science,71.868468,Sci&Tech,False,0,75.353724,Mkt&Fin,67.284781,25025.312840,True,2020-02-03 07:17:36.337038080,2020-08-01 04:33:07.830278912,3.0
3,3,Todd Edwards,F,44.676988,76.947303,Commerce,60.022930,Comm&Mgmt,False,0,94.473634,Mkt&HR,62.090481,NaN,False,NaT,NaT,12.0
4,4,Timothy Rice,F,73.255932,61.226478,Science,71.035746,Sci&Tech,False,0,59.232339,Mkt&HR,58.741125,32162.825874,True,2020-01-19 11:44:16.728017152,2020-09-18 11:20:09.256338432,3.0


In [18]:
new_data.name.isin(data.name).sum()

200

In order to solve this, we can pass an additional argument `anonymize_fields` to
our model when we create the instance.

This `anonymize_fields` argument will need to be a dictionary that contains:
- The name of the field that we want to anonymize.
- The category of the field that we want to use when we generate fake values for it.

The list complete list of possible categories can be seen in the [Faker Providers
](https://faker.readthedocs.io/en/master/providers.html) page, and it contains a huge
list of concepts such as:

- name
- address
- country
- city
- ssn
- credit_card_number
- credit_card_expier
- credit_card_security_code
- email
- telephone
- ...

In this case, since the field is an e-mail address, we will pass a dictionary indicating
the category `name`

In [19]:
model = GaussianCopula(
    primary_key='student_id',
    anonymize_fields={
        'name': 'name'
    }
)
model.fit(data)

As a result, we can see how the real `name` values have been replaced by other fake
names that were not taken from the real data that we learned.

In [20]:
new_data = model.sample(200)
new_data.head()

,student_id,name,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,Teresa Ryan,M,54.256003,77.615606,Commerce,63.051756,Comm&Mgmt,False,0,86.167858,Mkt&HR,59.828167,27934.882708,True,2020-01-11 22:07:05.575592448,2020-04-02 13:37:12.388929024,3.0
1,1,Monique Williams,M,68.777674,77.677627,Science,63.026272,Comm&Mgmt,False,1,69.912596,Mkt&HR,63.497502,40378.065476,True,2020-01-01 21:04:52.466290944,2020-10-03 11:01:04.867149568,12.0
2,2,Daniel Harrison,F,73.613542,52.495305,Science,65.726555,Comm&Mgmt,False,0,71.782044,Mkt&Fin,55.769197,22549.833667,True,2020-01-06 13:59:51.250882304,2020-03-29 15:45:03.941601536,3.0
3,3,Jill Graham,M,59.150170,60.929126,Commerce,61.275861,Comm&Mgmt,False,0,51.176688,Mkt&HR,54.188604,22532.337124,True,2020-01-02 19:49:49.849142528,2020-08-14 07:41:18.155750400,3.0
4,4,Tyler Mays,F,61.050104,62.028797,Commerce,67.557193,Sci&Tech,False,1,88.806534,Mkt&Fin,68.951671,NaN,False,NaT,NaT,6.0


In [21]:
new_data.name.isin(data.name).sum()

0

### Use different transformers

One thing that you may have noticed when executing the previous steps is that the fitting
process took much longer on the `student_placements_pii` dataset than it took on the
previous version that did not contain the sudent `name`s.

This happens because the `name` field is interpreted as a categorical variable, which the
Tabular Model [one-hot encoded](https://en.wikipedia.org/wiki/One-hot) generating 215 new
columns that it had to learn afterwards.

This transformation, which in this case was very inefficient, happens because the Tabular
Models apply [Reversible Data Transforms](https://github.com/sdv-dev/RDT) under the hood
to transform all the non-numerical variables, which the underlying models cannot handle,
into numerical representations which they can properly work with.

In the case of the `GaussianCopula`, the default transformation is a One-Hot encoding,
which can work very well with variables that have a little number of different values,
but which is very inefficient in cases where there is a large number of values.

For this reason, the Tabular Models have an additional argument called `field_transformers`
that let you select which transformer to apply to each column. This `field_transformers`
argument must be passed as a `dict` which contains the name of the fields for which
we want to use a transformer different than the default, and the name of the transformer
that we want to use.

Possible transformer names are:

* `integer`: Uses a `NumericalTransformer` of dtype `int`.
* `float`: Uses a `NumericalTransformer` of dtype `float`.
* `categorical`: Uses a `CategoricalTransformer` without gaussian noise.
* `categorical_fuzzy`: Uses a `CategoricalTransformer` adding gaussian noise.
* `one_hot_encoding`: Uses a `OneHotEncodingTransformer`.
* `label_encoding`: Uses a `LabelEncodingTransformer`.
* `boolean`: Uses a `BooleanTransformer`.
* `datetime`: Uses a `DatetimeTransformer`.

**NOTE**: For additional details about each one of the transformers, please visit [RDT](
https://github.com/sdv-dev/RDT)

Let's now try to improve the previous fitting process by changing the transformer
that we use for the `name` field to something other than the default.

As an example, we will use the `label_encoding` transformer, which instead of generating
one column for each possible value, it just replaces each value with a unique integer value.

In [22]:
model = GaussianCopula(
    primary_key='student_id',
    anonymize_fields={
        'name': 'name'
    },
    field_transformers={
        'name': 'label_encoding'
    }
)
model.fit(data)

In [23]:
new_data = model.sample(200)
new_data.head()

,student_id,name,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,0,Destiny Smith,M,66.695636,34.614727,Commerce,58.947420,Comm&Mgmt,False,0,50.519559,Mkt&HR,56.366306,20077.552474,True,2020-02-24 03:33:59.224180480,2020-09-12 22:28:20.817696512,3.0
1,1,James Smith,F,44.054187,61.576188,Commerce,70.931004,Comm&Mgmt,False,0,55.337901,Mkt&HR,54.308815,31493.674567,True,2020-01-05 05:36:08.200440576,2020-04-04 22:29:32.324812544,3.0
2,2,Joseph Mclaughlin,M,59.157425,69.371756,Commerce,65.065871,Comm&Mgmt,False,0,70.006051,Mkt&Fin,59.370396,NaN,False,NaT,NaT,6.0
3,3,Stephanie Mcdowell,M,57.413727,72.766079,Commerce,61.718940,Comm&Mgmt,False,1,88.453349,Mkt&HR,63.285239,NaN,False,NaT,NaT,12.0
4,4,Amy Gates,M,76.533955,72.549629,Science,76.073706,Comm&Mgmt,False,0,84.343519,Mkt&Fin,58.891185,21681.844305,True,2020-01-24 12:05:05.016603648,2020-10-13 21:33:49.999000064,3.0
